In [3]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess
import json
from evaluation.moses.metrics.utils  import logP, QED, SA
from rdkit import Chem
import pandas as pd
from rdkit import RDLogger 


In [4]:
def get_data(name, path='data/smiles/',return_smiles=False):
    splits={}
    RDLogger.EnableLog('rdApp.*')
    RDLogger.DisableLog('rdApp.*')  
    split_names=[ 'train1_pos_smiles','train1_neg_smiles','test_smiles','test_targets' ]
    for i,split in enumerate(split_names):
        exact_path=path+'{}/{}.txt'.format(name, split)
        #from data.smiles.carcinogens import test_smiles
        current_list = []
        with open(exact_path) as my_file:
         for line in my_file:
            current_list.append(line.strip())
        splits[split]=current_list     
    return splits
    

In [63]:
        
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
dataset_name='lipophilicity_astrazeneca'
splits=get_data(dataset_name ,return_smiles=True)


In [64]:
train1_pos_smiles,train1_neg_smiles, test_smiles, test_targets=splits['train1_pos_smiles'], splits['train1_neg_smiles'],splits['test_smiles'],splits['test_targets']
test_pos=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='1']
test_neg=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='0']
test_pos=[1,2,3,4,6,7,8,9,10,11,12]
test_neg=[1,2,3,4,5,6,7,9,9,10]

In [65]:

data=train1_pos_smiles+test_pos
mols =[Chem.MolFromSmiles(smiles)  for smiles in data]
log_p =[logP(mol) for mol in mols]
qeds= [QED(mol) for mol in mols]
sas= [SA(mol) for mol in mols]
d1={'smiles':data,'logP':log_p, 'qed':qeds,'SAS':sas}
trial=pd.DataFrame(data=d1)
trial.to_csv('{}_train1_pos.csv'.format(dataset_name), index=True)

In [66]:
data2=train1_neg_smiles+test_neg
mols =[Chem.MolFromSmiles(smiles)  for smiles in data2]
log_p =[logP(mol) for mol in mols]
qeds= [QED(mol) for mol in mols]
sas= [SA(mol) for mol in mols]
d2={'smiles':data2,'logP':log_p, 'qed':qeds,'SAS':sas}
trial=pd.DataFrame(data=d2)
trial.to_csv('{}_train1_neg.csv'.format(dataset_name), index=True)

In [68]:
valid_idx_pos=list(np.arange (len(train1_pos_smiles), len(train1_pos_smiles)+ len(test_pos)))
valid_idx_pos_=[str(i) for i in valid_idx_pos]
valid_idx_neg=list(np.arange (len(train1_neg_smiles), len(train1_neg_smiles)+ len(test_neg)))
valid_idx_neg_=[str(i) for i in valid_idx_neg]

In [69]:
dict={"valid_idxs":valid_idx_pos_}
with open("valid_idx_{}_train1_pos.json".format(dataset_name), "w") as outfile:
    json.dump(dict, outfile)


In [70]:
dict={"valid_idxs":valid_idx_neg_}
with open("valid_idx_{}_train1_neg.json".format(dataset_name), "w") as outfile:
    json.dump(dict, outfile)
